In [ ]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)

import h5py as h5

import tdg
import tdg.HMC as HMC
import tdg.plot as visualize

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
storage = 'analysis.h5'

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 5
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 8
beta = torch.tensor(1)
mu = torch.tensor(-1.5)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)
integrator = HMC.Omelyan(H, 20, 1)
hmc = HMC.MarkovChain(H, integrator)

Let's start from a configuration sampled from the "quenched" distribution, which ignores the fermion determinant.

In [ ]:
configurations = 1200

try:
    with h5.File(storage, 'r') as f:
        ensemble = tdg.ensemble.GrandCanonical.from_h5(f['/ensemble'])
    if len(ensemble) < configurations:
        raise
except:
    ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start='hot', progress=tqdm)
    with h5.File(storage, 'w') as f:
        ensemble.to_h5(f.create_group('/ensemble'))

We can visualize an observable.

In [ ]:
viz = visualize.History(3)
viz.plot(ensemble.S.real, 0)
viz.plot(ensemble.N('bosonic').real, 1)
viz.plot(ensemble.Spin(0).real, 1) # Another way of calculating N('fermionic')
viz.plot(ensemble.Spin(1).real, 2)
viz.plot(ensemble.Spin(2).real, 2)
viz.plot(ensemble.Spin(3).real, 2)

In [ ]:
from tdg.analysis import Binning

In [ ]:
binned = Binning(ensemble.cut(200).every(4), 4)

How many samples should be have, given that the ensemble started with 1200 configurations?

In [ ]:
expected = (
    (
        configurations-200 # cut
    )/4 # every
) / 4 # binning
print(f'The binning has {binned.bins} samples, while we expected {expected}')

In [ ]:
viz = visualize.ScatterTriangle(2)
viz.plot(
    (binned.N('bosonic'  ).real,
     binned.N('fermionic').real
    ))